In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import requests
import json
import os
import time

In [ ]:
#consts
dataset_path = "/content/gdrive/MyDrive/MIT_dataset/images/"

result_map = dict() 
face_id_map = dict()

url_base = "https://api.imagga.com/v2/faces/"
url_face_detection = url_base + "detections"
url_face_similarity = url_base + "similarity?face_id=%s&second_face_id=%s"

api_key = 'acc_d0e3bda2908fe1a'
api_secret = '2f7e705c0c8987e6321efb0d2e0721d7'

In [ ]:
def init():
  for filename in sorted(os.listdir(dataset_path)):
    #print(filename)
    key = filename.split('.')[0] #get file names without extension
    result_map.update({key : None}) #insert keys in map
    face_id_map.update({key : None}) #insert keys in map

In [ ]:
def detect_face(img_path):
  response = requests.post(
      url_face_detection, 
      auth=(api_key, api_secret), 
      files={'image': open(img_path, 'rb'), "return_face_id":1})
  
  response_json = response.json()
  
  #print("DETECT_FACE: ", json.dumps(response_json, indent = 4))
  return response_json["result"]["faces"][0]["face_id"]

In [ ]:
def add_face_id_to_map(file_extension = ".jpg"):
  global face_id_map
  for key in face_id_map.keys():
    complete_file_path = dataset_path + key + file_extension # ex: "/content/gdrive/MyDrive/MIT_dataset/images/0_1.jpg" 

    face_id = detect_face(complete_file_path)
    face_id_map.update({key : face_id})

  print(json.dumps(face_id_map, indent = 4)) #print face_id_map

In [ ]:
def check_similarity(face_id_1, face_id_2):

  time.sleep(5) #to make sure each request is successful 
  response = requests.get(
    url_face_similarity % (face_id_1, face_id_2),
    auth=(api_key, api_secret))

  response_json = response.json()

  if response_json["status"]["type"] == "success":
    #print("CHECK_SIMILARITY: ", response_json["result"]["score"])
    return "{:.2f}".format(response_json["result"]["score"]) #formating to be only 2 decimals
  elif response_json["status"]["type"] == "error":
    print("CHECK_SIMILARITY: ", response_json["status"]["text"]) #print errr message

In [ ]:
def add_results():
  global result_map, face_id_map

  for key1, value1 in face_id_map.items():

    person_confidence_score = dict()

    for key2, value2 in face_id_map.items():

      if value1 and value2 is not None:
        score = check_similarity(value1, value2)
        print("key1:", key1, " key2:", key2, " score:", score)
        person_confidence_score.update({key2 : score})

    if person_confidence_score :
      result_map.update({key1 : person_confidence_score})

  print(json.dumps(result_map, indent = 4))

In [ ]:
def show_results():
  print("========================================FINAL RESULTS========================================")
  for key, person_confidence_score_map in sorted (result_map.items()):
    score = None
    if person_confidence_score_map is not None:
      score = person_confidence_score_map.values()

    print(key, ":", score)

In [ ]:
init()
add_face_id_to_map()
add_results()
show_results()

{
    "0_1": "be37c4e63a95e56468cf96a1c0d3dd819aea80eb512f3fc73ae622eaa293028f",
    "0_2": "eea1cca48c06c52cdefe8afe6293cc70d24b7aa3863b881762e81157900b2485",
    "0_3": "621876aea8e66b667147dfbd5121eb339fb21305837140c7af49350749c3863b",
    "0_4": "5e576a5b44f9cc0aafbabd3564e2c4e1c0026555bfe63576174b53c347d3d888",
    "0_5": "0a96cb3b96f9ad8a3ed5214ebae944e6c68179896fa3343b5ebe789d01880c45",
    "1_1": "8d2abb54b579c25cbeea3958435ab6d82bef0d5de382fdac19cd60a7eef73f04",
    "1_2": "65c6089a66ad278dbb44fc64c3f55b2894f08605bd5dab9a092b570af1fb5cae",
    "1_3": "2768c503fcd93a0cff00288284e415ddeecc63c30f621b9caa8eab7307e997cb",
    "1_4": "8ecf3421e5112e5904b50b882064ffcb04e74c786ebfb8b9ad9be7f0a5018b82",
    "1_5": "ed2bbc123bbd827ad894e2beed232a8676e3dc46b38e065a90756aadd0db7fd0",
    "2_1": "41d2b5da741f0d539430ccb813d9ba862f55fefc1ce8e263a185cffc770f5bd6",
    "2_2": "1a9fdedc24fbe9f76dd9219ad7fabbafd68f8445e3714b46d926f861fd50adee",
    "2_3": "2a27c1cb0108683bb86cdb4871da8b2b3380ec